In [1]:
from lacbox.io import load_stats, load_oper
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

BB_PATH = Path.cwd().parent / 'my_design/BB_neo_turb_stats.csv'  # path to mean steady stats
STATS_PATH = Path.cwd().parent / 'dtu_10MW' / 'turb' / 'dtu_10mw_turb_stats.hdf5'  # path to mean steady stats
SUBFOLDER = 'tcb' 

In [2]:
# load the HAWC2 data from the stats file. Isolate the simulations with no tilt.
df, wsps = load_stats(STATS_PATH, subfolder=SUBFOLDER, statstype='turb')
df_BB, wsps_BB = load_stats(BB_PATH, subfolder=SUBFOLDER, statstype='turb')

SEEDS = 6



In [4]:
def extract_and_order_statistics(wsps, chan_df, seeds):
    """
    Extract and order:
     - wsp, max, mean, min for the points
     - wsp, mean_max, mean_mean, mean_min for the mean lines
    """

    # extract hawc2 wind and channel to plot from the HAWC2 stats
    val_wsp, val_max, val_mean, val_min = chan_df['wsp'], chan_df['max'], chan_df['mean'], chan_df['min']
    val_wsp, val_max, val_mean, val_min = np.array(val_wsp), np.array(val_max), np.array(val_mean), np.array(val_min)
    i_val = np.argsort(val_wsp)

    # Get mean of max/mean/min
    mean_max = np.average(val_max[i_val].reshape(-1, seeds), axis=1)
    mean_mean = np.average(val_mean[i_val].reshape(-1, seeds), axis=1)
    mean_min = np.average(val_min[i_val].reshape(-1, seeds), axis=1)

    points = {
        'wsp': val_wsp[i_val],
        'max': val_max[i_val],
        'mean': val_mean[i_val],
        'min': val_min[i_val]
    }

    lines = {
        'wsp': wsps,
        'max': mean_max,
        'mean': mean_mean,
        'min': mean_min
    }

    return points, lines

In [5]:
# isolate the tower clearance channel data
# chan_df = df.filter_channel(113, 'DLL inp   5')
chan_df = df.filter_channel(0, {0 :'DLL :  5 inpvec :   1  min. distance bladetips tower [m]'})

# Filter rows where 'desc' column matches the specific name
filtered_df = df[df['desc'] == 'DLL :  5 inpvec :   1  min. distance bladetips tower [m]']

# Display the filtered dataframe
# print(filtered_df)


tower_clearance = filtered_df['min']
# tower_clearance = chan_df_wtf['min']


tower_clearance_array = np.array(tower_clearance)

print(tower_clearance_array)

print(tower_clearance_array.min())


[14.35036659 11.99881935  9.2250185  13.32318497  7.91527271 11.19268703
 14.25718212 11.41734695 15.41445255  8.59133148 10.96473885  8.31780148
 12.36607933  8.32795238 13.98627949 10.48467922  9.02410507 15.16229534
 15.04970264 14.51843262 13.15967751 12.65079594 14.66797161 11.98375607
 11.19608593 15.99675655 13.49405861 10.54007149 14.0979023   8.93219376
 12.0998888  13.28540134 12.67029953  8.66289711 16.40191078  8.51593018
 14.08607864  8.45833969 13.99186802 14.36501503 15.13037395  8.32334232
 11.71794701 10.81178474 11.35311508  8.92172527  9.81187344 11.00609875
 15.36390209 14.13677979  8.79217911  7.84365797 13.41386127 10.39089108
  8.84844398 11.0133419  11.02552032 12.02308655 12.28248787 13.55341148
 11.48344612 13.20596695 13.67212391 14.80127716 13.94012833 12.61701298
 12.81168079 13.83052826 10.1453886  14.2454834  14.75996113 14.84742928
 14.37381077 15.45120335 15.07440758  9.71868706  8.99339199 15.8769474
 13.46093273 10.76733303 12.64439106 13.98094749 10.

In [109]:
CHAN_DESCS = {'TbFA': 'momentmx mbdy:tower nodenr:   1',
              'TbSS': 'momentmy mbdy:tower nodenr:   1',
              'YbTilt': 'momentmx mbdy:tower nodenr:  11',
              'YbRoll': 'momentmy mbdy:tower nodenr:  11',
              'ShftTrs': 'momentmz mbdy:shaft nodenr:   4',
              'OoPBRM': 'momentmx mbdy:blade1 nodenr:   1 coo: hub1',
              'IPBRM': 'momentmy mbdy:blade1 nodenr:   1 coo: hub1',
              'FlpBRM': 'momentmx mbdy:blade1 nodenr:   1 coo: blade1',
              'EdgBRM': 'momentmy mbdy:blade1 nodenr:   1 coo: blade1',
              'OoPHub': 'momentmx mbdy:hub1 nodenr:   1 coo: hub1',
              'IPHub': 'momentmy mbdy:hub1 nodenr:   1 coo: hub1',
              }

# what channels we want to plot
chan_ids = ['TbFA', 'TbSS',
            'YbTilt', 'YbRoll', 'ShftTrs', 'OoPBRM', 'IPBRM']

max_dict = {}
min_dict = {}
mean_dict = {}
for iplot, chan_id in enumerate(chan_ids):
    
    chan_df = df.filter_channel(chan_id, CHAN_DESCS)
    
    # Doing this works for all channel, but not the tip clearance, because f me
    # channel_wsp = chan_df['wsp']
    # channel_max = chan_df['max']
    # channel_min = chan_df['min']
    # channel_mean = chan_df['mean']
    
    points_BB, lines_BB = extract_and_order_statistics(wsps_BB, chan_df, SEEDS)
    
    max_val = np.array(lines_BB['max']).max()
    min_val = np.array(lines_BB['min']).min()
    mean_val = np.array(lines_BB['mean']).mean()
    
    # max_val = np.array(channel_mean).max()
    max_dict[chan_id]  =  max_val * 1.35 * 1.25
    min_dict[chan_id]  =  min_val * 1.35 * 1.25
    mean_dict[chan_id] = mean_val

print(max_dict)
print(min_dict)
print(mean_dict)


# Initialize the new dictionary for absolute maximum values
absolute_max_dict = {}

# Loop through each key in max_dict (assuming keys in max_dict and min_dict are identical)
for key in max_dict.keys():
    # Calculate the absolute maximum value between max_dict and min_dict for each key
    absolute_max_dict[key] = max(abs(max_dict[key]), abs(min_dict[key]))

# Print the resulting dictionary with absolute maximum values
print(absolute_max_dict)


{'TbFA': 364297.47363281256, 'TbSS': 117720.31530761719, 'YbTilt': 54987.89062500001, 'YbRoll': 23907.457946777344, 'ShftTrs': -685.476744890213, 'OoPBRM': 27557.119445800785, 'IPBRM': 40770.70642089844}
{'TbFA': -51051.62658691406, 'TbSS': -51982.17736816406, 'YbTilt': -38194.008178710945, 'YbRoll': 239.45605301856997, 'ShftTrs': -20365.171325683594, 'OoPBRM': -72275.13830566406, 'IPBRM': -27472.3678894043}
{'TbFA': 96549.52281564193, 'TbSS': 12733.729012465428, 'YbTilt': 1867.128824448829, 'YbRoll': 9102.224890923591, 'ShftTrs': -8842.391787113766, 'OoPBRM': -15499.956871376378, 'IPBRM': 2753.3949561108216}
{'TbFA': 364297.47363281256, 'TbSS': 117720.31530761719, 'YbTilt': 54987.89062500001, 'YbRoll': 23907.457946777344, 'ShftTrs': 20365.171325683594, 'OoPBRM': 72275.13830566406, 'IPBRM': 40770.70642089844}
